In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [36]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import plot_model
import numpy as np
from glob import glob
import matplotlib.pyplot as plt



In [37]:
import glob
import pandas as pd
import os

In [4]:
filepath=[]
label =[]
for i in glob.glob('/content/drive/MyDrive/Files/Capstone_project/'+'*/*.png'):
  filepath.append(i)
  label.append(i.split('/')[-2])

In [6]:
data = pd.DataFrame({"Filepath":filepath,'Label':label})

In [7]:
data

,Filepath,Label
0,/content/drive/MyDrive/Files/Capstone_project/...,non-COVID
1,/content/drive/MyDrive/Files/Capstone_project/...,non-COVID
2,/content/drive/MyDrive/Files/Capstone_project/...,non-COVID
3,/content/drive/MyDrive/Files/Capstone_project/...,non-COVID
4,/content/drive/MyDrive/Files/Capstone_project/...,non-COVID
...,...,...
2475,/content/drive/MyDrive/Files/Capstone_project/...,COVID
2476,/content/drive/MyDrive/Files/Capstone_project/...,COVID
2477,/content/drive/MyDrive/Files/Capstone_project/...,COVID
2478,/content/drive/MyDrive/Files/Capstone_project/...,COVID


In [8]:
data['Label'].value_counts()

COVID        1251
non-COVID    1229
Name: Label, dtype: int64

In [9]:
master_data=data.sample(frac=1)

In [10]:
master_data.reset_index(inplace=True)
master_data.drop(['index'],axis=1,inplace=True)

In [ ]:
master_data

,Filepath,Label
0,/content/drive/MyDrive/Files/Capstone_project/...,COVID
1,/content/drive/MyDrive/Files/Capstone_project/...,non-COVID
2,/content/drive/MyDrive/Files/Capstone_project/...,COVID
3,/content/drive/MyDrive/Files/Capstone_project/...,COVID
4,/content/drive/MyDrive/Files/Capstone_project/...,non-COVID
...,...,...
2475,/content/drive/MyDrive/Files/Capstone_project/...,COVID
2476,/content/drive/MyDrive/Files/Capstone_project/...,COVID
2477,/content/drive/MyDrive/Files/Capstone_project/...,non-COVID
2478,/content/drive/MyDrive/Files/Capstone_project/...,COVID


In [11]:
from keras.models import Sequential,Model
from keras.layers import Dense,Flatten,Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint,EarlyStopping

In [12]:
train_generator = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    width_shift_range=0.2,
    height_shift_range=0.2,
    validation_split=0.2
)

In [13]:
test_generator = ImageDataGenerator(
    rescale=1./255
)

In [14]:
master_data['Label'] = master_data['Label'].replace({"COVID":0,"non-COVID":1})

In [15]:
master_data['Label'].unique()

array([0, 1])

In [16]:
master_data.head(2)

,Filepath,Label
0,/content/drive/MyDrive/Files/Capstone_project/...,0
1,/content/drive/MyDrive/Files/Capstone_project/...,0


In [17]:
train_images = train_generator.flow_from_dataframe(
    dataframe=master_data,
    x_col='Filepath',
    y_col='Label',
    target_size=(120,120),
    color_mode='rgb',
    class_mode='raw',
    batch_size=4,
    shuffle=True,
    subset='training'
)

Found 1984 validated image filenames.


In [18]:
val_images = train_generator.flow_from_dataframe(
    dataframe=master_data,
    x_col='Filepath',
    y_col='Label',
    target_size=(120, 120),
    color_mode='rgb',
    class_mode='raw',
    batch_size=4,
    shuffle=True,
    subset='validation'
)

Found 496 validated image filenames.


In [19]:
resnet = ResNet50(
    input_shape = (120,120,3), 
    weights = None, 
    include_top = False   
)

In [20]:
for layer in resnet.layers:
    layer.trainable = True

In [21]:
x = Flatten()(resnet.output)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)

In [22]:
prediction = Dense(1, activation = 'sigmoid')(x)

In [23]:
model = Model(inputs = resnet.input, outputs = prediction)

In [24]:
model.compile (
    loss = 'binary_crossentropy',
    optimizer = 'adam',
    metrics = ['accuracy']
)

In [25]:
#Callbacks

checkpoint = ModelCheckpoint("Model.h5", monitor='val_loss', save_best_only=True, save_weights_only=False, mode='auto')
early = EarlyStopping(patience=7, verbose=2, mode='auto')


In [27]:
history = model.fit_generator(
    train_images,
    validation_data = val_images,
    epochs = 70,
    steps_per_epoch = len(train_images),
    validation_steps = len(val_images),
    callbacks = [checkpoint, early],
    verbose=2)


Epoch 1/70


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


496/496 - 503s - loss: 0.7557 - accuracy: 0.6557 - val_loss: 0.5154 - val_accuracy: 0.7540 - 503s/epoch - 1s/step
Epoch 2/70
496/496 - 29s - loss: 0.6334 - accuracy: 0.6840 - val_loss: 0.6253 - val_accuracy: 0.6290 - 29s/epoch - 59ms/step
Epoch 3/70
496/496 - 32s - loss: 0.6122 - accuracy: 0.6784 - val_loss: 0.4715 - val_accuracy: 0.7702 - 32s/epoch - 65ms/step
Epoch 4/70
496/496 - 29s - loss: 0.6104 - accuracy: 0.6996 - val_loss: 0.8816 - val_accuracy: 0.6512 - 29s/epoch - 58ms/step
Epoch 5/70
496/496 - 32s - loss: 0.5740 - accuracy: 0.7324 - val_loss: 0.4665 - val_accuracy: 0.7661 - 32s/epoch - 65ms/step
Epoch 6/70
496/496 - 30s - loss: 0.5176 - accuracy: 0.7611 - val_loss: 0.4932 - val_accuracy: 0.7903 - 30s/epoch - 61ms/step
Epoch 7/70
496/496 - 31s - loss: 0.5241 - accuracy: 0.7676 - val_loss: 0.3795 - val_accuracy: 0.8589 - 31s/epoch - 63ms/step
Epoch 8/70
496/496 - 29s - loss: 0.5115 - accuracy: 0.7656 - val_loss: 0.4636 - val_accuracy: 0.7964 - 29s/epoch - 58ms/step
Epoch 9/70
